In [2]:
import os
from pathlib import Path
import pylidc as pl
from utils import cluster_annots, get_cropped_annot
import SimpleITK as sitk
from radiomics import featureextractor
import pandas as pd

### 1. Radiomics

**Dani, please run after downloading dicoms**

In [ ]:
#requirements
%pip install SimpleITK

Note: you may need to restart the kernel to use updated packages.


- Get Cropped Images and Maks from Scans

> We began by setting up a directory to store data, creating folders for each patient and each nodule.

> For each scan, we checked if the annotations for that scan exist, and if so, we used the **cluster_annots()** function to group similar annotations and the **get_cropped_annot()** function to obtain cropped images of the nodules and their respective masks.


> The cropped data is converted from NumPy arrays to SimplesITK images and then saved in NIfTI (.nii) format, which supports 3D medical imaging

**Note:** The decision to use 3D instead of 2D feature extraction is driven by the fact that lung nodules are three-dimensional structures, and capturing their full spatial characteristics seems important for accurate analysis.

In [ ]:
data_path = Path('../data')
os.makedirs(data_path, exist_ok=True)

#import all scans
#scans = pl.query(pl.Scan).filter(pl.Scan.patient_id == 'LIDC-IDRI-0001').all()
scans = pl.query(pl.Scan).all()
for scan in scans:
    #print(scan.patient_id)
    patient_dir = Path('../data/')/scan.patient_id
    os.makedirs(patient_dir, exist_ok=True)
    try:
        if len(scan.annotations) == 0:
            # Scan has no annotations, there is nothing to do
            continue
        
        nodules = cluster_annots(scan)
        croped, mask = get_cropped_annot(scan, nodules, False)

    except RuntimeError as e:
        print(e)
    
    #save images and maks
    for i in range(len(mask)):
        nodule_id = nodules[i][0].id
        #print(nodule_id)
        nod_path = Path(f'../data/{scan.patient_id}/Nodule_{i}')
        os.makedirs(nod_path, exist_ok=True)
        image_path = Path(f'{nod_path}/image')
        mask_path = Path(f'{nod_path}/mask')
        os.makedirs(image_path, exist_ok=True)
        os.makedirs(mask_path, exist_ok=True)

        #numpy array to SimpleITK image
        mask_sitk = sitk.GetImageFromArray(mask[i].astype(int))
        image_sitk = sitk.GetImageFromArray(croped[i])

        #save SimpleITK images as.nii (suports 3D)
        mask_path = Path(f'{mask_path}/mask.nii')
        image_path = Path(f'{image_path}/image.nii')
        sitk.WriteImage(mask_sitk, mask_path)
        sitk.WriteImage(image_sitk, image_path)

84


- Extract Features using pyradiomics library


> This function processes all nodules for a given patient. For each nodule, the image and mask files are loaded using SImpleITK, and radiomics features are extracted by the **RadiomicsFeatureExatractor()**. Then, the extracted features are converted into a dataframe.

In [ ]:
def extract_features_from_patient(patient_dir):
    extractor = featureextractor.RadiomicsFeatureExtractor()
    patient_id = os.path.basename(patient_dir)
    
    patient_results = []
    
    for nodule_dir in os.listdir(patient_dir):
        nodule_path = os.path.join(patient_dir, nodule_dir)

        image_path = os.path.join(nodule_path, 'image', 'image.nii')
        mask_path = os.path.join(nodule_path, 'mask', 'mask.nii')

        #verify if the files exist and import them
        if os.path.exists(image_path) and os.path.exists(mask_path):
            image = sitk.ReadImage(image_path)
            mask = sitk.ReadImage(mask_path)

            features = extractor.execute(image, mask)

            #convert to dictionary
            feature_dict = dict(features)
            feature_dict['patient_id'] = patient_id
            feature_dict['nodule_id'] = nodule_id

            patient_results.append(feature_dict)

    df = pd.DataFrame(patient_results)

    return df

In [ ]:
radiomics_df = pd.DataFrame()

for patient_dir in os.listdir('../data'):
    patient_dir = os.path.join('../data',patient_dir)
    patient_df = extract_features_from_patient(patient_dir)

    #remove libraries versions, configuration details, hashs, ...
    patient_df = patient_df[patient_df.columns.drop(list(patient_df.filter(regex='Versions')))]
    patient_df = patient_df[patient_df.columns.drop(list(patient_df.filter(regex='Configuration')))]
    patient_df = patient_df[patient_df.columns.drop(list(patient_df.filter(regex='Hash')))]
    patient_df = patient_df[patient_df.columns.drop(list(patient_df.filter(regex='diagnostics')))]

    radiomics_df = pd.concat([radiomics_df, patient_df], ignore_index=True)

radiomics_df.to_csv('radiomics.csv')
radiomics_df

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,original_shape_MinorAxisLength,...,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength,patient_id,nodule_id
0,0.97256,0.242197,7.854413,32.429873,44.01136216933077,45.617978911828175,35.22782990761707,48.45616575834287,5382.208333333333,31.539994,...,-3.203426503814917e-16,0.00018422991893883567,0.0,0.0,1000000.0,0.0,0.0,0.0,LIDC-IDRI-0001,84
